In [1]:
import matplotlib.pyplot as plt
import math
from tqdm import tqdm
from qutip import *
from device import ChalmersQubits
from qutip_qip.circuit import QubitCircuit, Gate
from qutip_qip.operations import rz, rx, ry
from qutip_qip.compiler import Scheduler, Instruction
from device.operations import *
import numpy as np
%load_ext autoreload
%autoreload 2

In [2]:
def modified_pauli_matrices():
    modified_pauli = []
    # Identity
    modified_pauli.append(Qobj(np.array([[1, 0, 0], [0, 1, 0], [0, 0, 0]])))
    # Sigma X
    modified_pauli.append(Qobj(np.array([[0, 1, 0], [1, 0, 0], [0, 0, 0]])))
    # Sigma Y
    modified_pauli.append(Qobj(np.array([[0, -1j, 0], [1j, 0, 0], [0, 0, 0]])))
    # Sigma Z
    modified_pauli.append(Qobj(np.array([[1, 0, 0], [0, -1, 0], [0, 0, 0]])))
    return modified_pauli

modified_pauli = modified_pauli_matrices()

In [3]:
def all_possible_combinations(matrices, num_qutrits):
    if num_qutrits == 1:
        return matrices
    else:
        prev_combinations = all_possible_combinations(matrices, num_qutrits - 1)
        new_combinations = []
        for m1 in matrices:
            for m2 in prev_combinations:
                new_combinations.append(tensor(m1, m2))
        return new_combinations

# Average Gate Fidelity 3 Qubit Gate

In [23]:
# Ideal gate
def cczs(args):
    theta, phi, gamma = args
    U = np.array([[1, 0, 0, 0, 0, 0, 0, 0],
                  [0, 1, 0, 0, 0, 0, 0, 0],
                  [0, 0, 1, 0, 0, 0, 0, 0],
                  [0, 0, 0, 1, 0, 0, 0, 0],
                  [0, 0, 0, 0, 1, 0, 0, 0],
                  [0, 0, 0, 0, 0, -np.exp(-1j*gamma)*np.sin(theta/2)**2 + np.cos(theta/2)**2, 
                                    (1/2)*(1 + np.exp(-1j*gamma))*np.exp(-1j*phi)*np.sin(theta), 0],
                  [0, 0, 0, 0, 0, (1/2)*(1 + np.exp(-1j*gamma))*np.exp(1j*phi)*np.sin(theta), 
                                    -np.exp(-1j*gamma)*np.cos(theta/2)**2 + np.sin(theta/2)**2, 0],
                  [0, 0, 0, 0, 0, 0, 0, -np.exp(1j*gamma)]], dtype="complex")
    return Qobj(U, dims=[[2]*3, [2]*3])

In [24]:
# Define a circuit and run the simulation
num_qubits = 3

# Dimension
d = 2**num_qubits

# Create all combinations of Paulis
all_paulis = all_possible_combinations(modified_pauli, num_qubits)

# This is the total number of combinations
num_paulis = len(all_paulis)

# Quantum circuit
circuit = QubitCircuit(num_qubits)
circuit.user_gates = {"CCZS": cczs}

# CCZS
circuit.add_gate("CCZS", targets=[0,1,2], arg_value=[np.pi/2,np.pi/2,0])

# Compute the ideal unitary from the circuit
U = circuit.compute_unitary()
U

Quantum object: dims = [[2, 2, 2], [2, 2, 2]], shape = (8, 8), type = oper, isherm = True
Qobj data =
[[ 1.+0.j  0.+0.j  0.+0.j  0.+0.j  0.+0.j  0.+0.j  0.+0.j  0.+0.j]
 [ 0.+0.j  1.+0.j  0.+0.j  0.+0.j  0.+0.j  0.+0.j  0.+0.j  0.+0.j]
 [ 0.+0.j  0.+0.j  1.+0.j  0.+0.j  0.+0.j  0.+0.j  0.+0.j  0.+0.j]
 [ 0.+0.j  0.+0.j  0.+0.j  1.+0.j  0.+0.j  0.+0.j  0.+0.j  0.+0.j]
 [ 0.+0.j  0.+0.j  0.+0.j  0.+0.j  1.+0.j  0.+0.j  0.+0.j  0.+0.j]
 [ 0.+0.j  0.+0.j  0.+0.j  0.+0.j  0.+0.j  0.+0.j  0.-1.j  0.+0.j]
 [ 0.+0.j  0.+0.j  0.+0.j  0.+0.j  0.+0.j  0.+1.j  0.+0.j  0.+0.j]
 [ 0.+0.j  0.+0.j  0.+0.j  0.+0.j  0.+0.j  0.+0.j  0.+0.j -1.+0.j]]

In [25]:
t1 = 40 * 10**3 # nano seconds
t2 = 60 * 10**3 # nano seconds
# Create quantum device
myprocessor = ChalmersQubits(num_qubits, t1=t1, t2=t2)
# Load circuit onto device
tlist, coeffs = myprocessor.load_circuit(circuit)

In [26]:
# average gate fidelity
F = 0
# compute the average gate fidelity
for idx, pauli in enumerate(tqdm(all_paulis, desc="Computing average gate fidelity", unit="pauli operator")):
    # Run master equation simulation
    rho = myprocessor.run_state(pauli)
    # Project the modified Pauli onto the qubit subspace
    P = project_on_qubit(pauli)
    # Project the final state onto the qubit subspace
    M = project_on_qubit(rho.states[-1])
    # Sum up the measurements to get the avg gate fid
    F += ((U*P.dag()*U.dag()*M).tr())

Computing average gate fidelity: 100%|█| 64/64 [07:45<00:00,  7.27s/pauli operat


Average gate fidelity with noise

In [31]:
avg_gate_fid = ((F+d**2)/(d**2*(d+1))).real
print("Average gate fidelity",round(avg_gate_fid*100,2))

Average gate fidelity 98.02


Average gate fidelity with no noise

In [28]:
rho = myprocessor.run_propagator(noisy=False)
M = project_on_qubit(rho[-1])

In [29]:
print("Noiseless Average gate fidelity",round(average_gate_fidelity(U,M)*100,2))

Noiseless Average gate fidelity 100.0


# Average Gate Fidelity 2 Qubit Gate

In [18]:
# Define a circuit and run the simulation
num_qubits = 2

# Dimension
d = 2**num_qubits

# Create all combinations of Paulis
all_paulis = all_possible_combinations(modified_pauli, num_qubits)

# This is the total number of combinations
num_paulis = len(all_paulis)

# Quantum circuit
circuit = QubitCircuit(num_qubits)

# CZ
circuit.add_gate("CZ", targets=[1], controls=[0])

# Compute the ideal unitary from the circuit
U = circuit.compute_unitary()
U

Quantum object: dims = [[2, 2], [2, 2]], shape = (4, 4), type = oper, isherm = True
Qobj data =
[[ 1.  0.  0.  0.]
 [ 0.  1.  0.  0.]
 [ 0.  0.  1.  0.]
 [ 0.  0.  0. -1.]]

In [19]:
t1 = 40 * 10**3 # nano seconds
t2 = 60 * 10**3 # nano seconds
# Create quantum device
myprocessor = ChalmersQubits(num_qubits, t1=t1, t2=t2)
# Load circuit onto device
tlist, coeffs = myprocessor.load_circuit(circuit)

In [20]:
# average gate fidelity
F = 0
# compute the average gate fidelity
for idx, pauli in enumerate(tqdm(all_paulis, desc="Computing average gate fidelity", unit="pauli operator")):
    # Run master equation simulation
    rho = myprocessor.run_state(pauli)
    # Project the modified Pauli onto the qubit subspace
    P = project_on_qubit(pauli)
    # Project the final state onto the qubit subspace
    M = project_on_qubit(rho.states[-1])
    # Sum up the measurements to get the avg gate fid
    F += ((U*P.dag()*U.dag()*M).tr())

Computing average gate fidelity: 100%|█| 16/16 [00:15<00:00,  1.04pauli operator


In [21]:
avg_gate_fid = ((F+d**2)/(d**2*(d+1))).real
print("Average gate fidelity",round(avg_gate_fid*100,2))

Average gate fidelity 98.39


In [22]:
rho = myprocessor.run_propagator(noisy=False)
M = project_on_qubit(rho[-1])
print("Noiseless Average gate fidelity",round(average_gate_fidelity(U,M)*100,2))

Noiseless Average gate fidelity 100.0


# Average Gate Fidelity 1 Qubit Gate

In [32]:
# Define a circuit and run the simulation
num_qubits = 1

# Dimension
d = 2**num_qubits

# Create all combinations of Paulis
all_paulis = all_possible_combinations(modified_pauli, num_qubits)

# This is the total number of combinations
num_paulis = len(all_paulis)

# Quantum circuit
circuit = QubitCircuit(num_qubits)

# RX
circuit.add_gate("RX", targets=[0], arg_value=np.pi)

# Compute the ideal unitary from the circuit
U = circuit.compute_unitary()
U

Quantum object: dims = [[2], [2]], shape = (2, 2), type = oper, isherm = False
Qobj data =
[[0.+0.j 0.-1.j]
 [0.-1.j 0.+0.j]]

In [33]:
t1 = 40 * 10**3 # nano seconds
t2 = 60 * 10**3 # nano seconds
# Create quantum device
myprocessor = ChalmersQubits(num_qubits, t1=t1, t2=t2)
# Load circuit onto device
tlist, coeffs = myprocessor.load_circuit(circuit)

In [34]:
# average gate fidelity
F = 0
# compute the average gate fidelity
for idx, pauli in enumerate(tqdm(all_paulis, desc="Computing average gate fidelity", unit="pauli operator")):
    # Run master equation simulation
    rho = myprocessor.run_state(pauli)
    # Project the modified Pauli onto the qubit subspace
    P = project_on_qubit(pauli)
    # Project the final state onto the qubit subspace
    M = project_on_qubit(rho.states[-1])
    # Sum up the measurements to get the avg gate fid
    F += ((U*P.dag()*U.dag()*M).tr())

Computing average gate fidelity: 100%|█| 4/4 [00:00<00:00,  4.55pauli operator/s


In [36]:
rho = myprocessor.run_propagator(noisy=False)
M = project_on_qubit(rho[-1])
print("Noiseless Average gate fidelity",round(average_gate_fidelity(U,M)*100,4))

Noiseless Average gate fidelity 99.9996


In [37]:
2 * np.pi * 1 * 1e-3

0.006283185307179587